In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
import numpy as np

#obtain PER rankings data
r = requests.get('https://www.basketball-reference.com/leaders/per_active.html')
root = BeautifulSoup(r.content,'html5lib')

# Get table data
table = root.find('table')

In [2]:
#Create PER Table
per_table = pd.DataFrame(columns=['Rank','Player','Career PER'], index = range(0,100))

# Put data in table
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    row_marker += 1
    for column in columns:
        if (row_marker > 1):
            per_table.iat[row_marker-2,column_marker] = column.get_text().strip()
            column_marker += 1

#We dont need the rank
del per_table['Rank']

In [3]:
links = []

#Obtain links of each players to scrape more data per players
for link in BeautifulSoup(r.content, 'html.parser', parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        if link['href'] in str(table) and '.html' in link['href']:
            location = "http://www.basketball-reference.com" + link['href']
            if location not in links:
                 links.append(location)

per_table["Link"] = links
per_table

,Player,Career PER,Link
0,LeBron James,27.70,http://www.basketball-reference.com/players/j/...
1,Chris Paul,25.71,http://www.basketball-reference.com/players/p/...
2,Kevin Durant,25.21,http://www.basketball-reference.com/players/d/...
3,Dwyane Wade,24.19,http://www.basketball-reference.com/players/w/...
4,Russell Westbrook,23.72,http://www.basketball-reference.com/players/w/...
5,Stephen Curry,23.57,http://www.basketball-reference.com/players/c/...
6,James Harden,23.26,http://www.basketball-reference.com/players/h/...
7,Dirk Nowitzki,22.77,http://www.basketball-reference.com/players/n/...
8,Blake Griffin,22.68,http://www.basketball-reference.com/players/g/...
9,DeMarcus Cousins,22.49,http://www.basketball-reference.com/players/c/...


In [4]:
rand = per_table.sample(1)

for link in rand["Link"]:
#     link += "#advanced_pbp::none"    
    print(link)
    
    #obtain PER rankings data
    req = requests.get(link)
    rt = BeautifulSoup(req.content,'html5lib')

    # Get table data
    playByPlay_table = rt.find('table', id="advanced_pbp")

    print(playByPlay_table)

http://www.basketball-reference.com/players/c/collida01.html
None


In [7]:
table

<table class="suppress_glossary suppress_csv sortable stats_table" data-cols-to-freeze="0" id="stats_active_per"><caption> Table</caption><thead><tr>
<th>Rank</th>
<th>Player</th>
<th>PER</th>
</tr></thead>
<tbody><tr>
<td>1.</td>
<td><strong><a href="/players/j/jamesle01.html">LeBron James</a>
</strong></td><td>27.70</td>
</tr>
<tr>
<td>2.</td>
<td><strong><a href="/players/p/paulch01.html">Chris Paul</a>
</strong></td><td>25.71</td>
</tr>
<tr>
<td>3.</td>
<td><strong><a href="/players/d/duranke01.html">Kevin Durant</a>
</strong></td><td>25.21</td>
</tr>
<tr>
<td>4.</td>
<td><strong><a href="/players/w/wadedw01.html">Dwyane Wade</a>
</strong></td><td>24.19</td>
</tr>
<tr>
<td>5.</td>
<td><strong><a href="/players/w/westbru01.html">Russell Westbrook</a>
</strong></td><td>23.72</td>
</tr>
<tr>
<td>6.</td>
<td><strong><a href="/players/c/curryst01.html">Stephen Curry</a>
</strong></td><td>23.57</td>
</tr>
<tr>
<td>7.</td>
<td><strong><a href="/players/h/hardeja01.html">James Harden</a>
<